In [1]:
import pandas as pd
import numpy as np
from loaddata import GetBookData
from surprise import SVD, accuracy, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering 
from surprise.model_selection import cross_validate, KFold
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset
from surprise.model_selection import GridSearchCV
import heapq
from collections import defaultdict
from operator import itemgetter

In [2]:
items = pd.read_csv('C:/Users/Jon/Documents/Statistics/Machine Learning/Project/Book Crossing/BX Project/BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
items.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
users = pd.read_csv('C:/Users/Jon/Documents/Statistics/Machine Learning/Project/Book Crossing/BX Project/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']
users['userID'] = users['userID'].apply(lambda x: str(x))
ratings = pd.read_csv('C:/Users/Jon/Documents/Statistics/Machine Learning/Project/Book Crossing/BX Project/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']
ratings['userID'] = ratings['userID'].apply(lambda x: str(x))

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\Jon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:

In [3]:
data = GetBookData(items,ratings,users)

In [4]:
trainSet = data.build_full_trainset()

In [5]:
sim_options = {'name': 'cosine', 'user_based': True}

In [6]:
model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [7]:
pd.DataFrame(simsMatrix)

,0,1,2,3,4,5,6,7,8,9,...,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116
0,1.000000,1.0,0.000000,0.0,1.000000,0.948683,0.0,0.00000,0.0,0.000000,...,1.000000,0.0,1.000000,0.978913,0.000000,1.0,1.000000,1.000000,0.0,0.998274
1,1.000000,1.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,...,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,0.000000,0.0,1.000000,0.0,1.000000,0.000000,0.0,0.00000,0.0,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.998618
3,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.0,1.00000,0.0,0.000000,...,0.000000,0.0,0.000000,1.000000,0.984387,0.0,1.000000,1.000000,0.0,0.000000
4,1.000000,0.0,1.000000,0.0,1.000000,1.000000,0.0,0.00000,0.0,1.000000,...,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.998698,0.998618,1.0,0.997785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.000000
1113,1.000000,0.0,0.000000,1.0,0.998698,0.000000,0.0,0.00000,0.0,0.000000,...,0.000000,1.0,0.988985,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.000000
1114,1.000000,0.0,0.000000,1.0,0.998618,1.000000,0.0,0.99146,0.0,0.000000,...,0.000000,1.0,1.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.0,1.000000
1115,0.000000,0.0,0.000000,0.0,1.000000,0.964764,0.0,0.00000,0.0,0.999838,...,1.000000,0.0,0.962669,1.000000,0.000000,0.0,0.000000,0.000000,1.0,0.000000


In [8]:
# Get top N similar users to our test subject
testSubject = '277427'
testUserInnerID = trainSet.to_inner_uid(testSubject)
similarityRow = simsMatrix[testUserInnerID]

In [9]:
similarUsers = []
for innerID, score in enumerate(similarityRow):
    if (innerID != testUserInnerID):
        similarUsers.append( (innerID, score) )


In [10]:
k = 10
kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])

In [ ]:
kNeighbors

In [11]:
# Get the stuff they rated, and add up ratings for each item, weighted by user similarity
candidates = defaultdict(float)
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1]
    theirRatings = trainSet.ur[innerID]
    for rating in theirRatings:
        candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore

In [12]:
# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1

In [13]:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(items[items['ISBN'] == trainSet.to_raw_iid(itemID)]['bookTitle'], ratingSum)
        pos += 1
        if (pos > 10):
            break

408    The Lovely Bones: A Novel
Name: bookTitle, dtype: object 6.6
1922    Snow Falling on Cedars
Name: bookTitle, dtype: object 5.2
647    Angela's Ashes: A Memoir
Name: bookTitle, dtype: object 5.2
1306    The Color of Water: A Black Man's Tribute to H...
Name: bookTitle, dtype: object 5.0
37    To Kill a Mockingbird
Name: bookTitle, dtype: object 4.0
1284    Lamb : The Gospel According to Biff, Christ's ...
Name: bookTitle, dtype: object 4.0
5070    Bridget Jones's Diary
Name: bookTitle, dtype: object 3.8
2056    Sister of My Heart
Name: bookTitle, dtype: object 3.6
3482    The Curious Incident of the Dog in the Night-T...
Name: bookTitle, dtype: object 3.6
1702    The Reader
Name: bookTitle, dtype: object 3.6
2995    Song of Solomon (Oprah's Book Club (Paperback))
Name: bookTitle, dtype: object 3.4000000000000004


In [26]:
pd.merge(ratings[ratings['userID'] == '277427'],items[
    ['ISBN','bookTitle']], on='ISBN', how = 'left').sort_values(by = 'bookRating',  ascending=False)

,userID,ISBN,bookRating,bookTitle
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...
384,277427,0836213076,10,Its Obvious You Wont Survive By Your Wit
391,277427,0836218817,10,Unnatural Selections
393,277427,0836221192,10,Dilbert Fugitive From The Cubicle Police
396,277427,0836236688,10,Dilbert: Seven Years Of Highly Defective Peopl...
...,...,...,...,...
177,277427,044011585X,0,Crossings
176,277427,0440113423,0,"The Colonists (Australians, No 6)"
175,277427,043936213X,0,Harry Potter and the Sorcerer's Stone (Book 1)
174,277427,0439136369,0,Harry Potter and the Prisoner of Azkaban (Book 3)


In [19]:
items[['ISBN','bookTitle']]

,ISBN,bookTitle
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi
...,...,...
271355,0440400988,There's a Bat in Bunk Five
271356,0525447644,From One to One Hundred
271357,006008667X,Lily Dale : The True Story of the Town that Ta...
271358,0192126040,Republic (World's Classics)


In [ ]:
item = items[items['ISBN'] == trainSet.to_raw_iid(176)].bookTitle.str.get()

In [ ]:
items[items['ISBN'] == trainSet.to_raw_iid(176)].bookTitle

In [ ]:
items[items['ISBN'] == trainSet.to_raw_iid(176)].bookTitle